In [37]:
BETA = 0
as = [0, 1.1 , 2.1] #prices public info
cs = [1, 2] #costs, private info
n= 2

function init_Ws(as,cs,n, W0)
    Ws = Dict()
    argumento=fill(as, n)
    cartesian_as=collect(Iterators.product(argumento...))

    for c=cs
        for a=cartesian_as
                Ws["$(a),$(c)"] = (W0,0)
        end
    end
    return Ws
end

W0 = init_Ws(as,cs,2,10)


Dict{Any,Any} with 18 entries:
  "(2.1, 2.1),1" => (10, 0)
  "(0.0, 2.1),2" => (10, 0)
  "(1.1, 2.1),1" => (10, 0)
  "(0.0, 0.0),2" => (10, 0)
  "(1.1, 0.0),1" => (10, 0)
  "(1.1, 1.1),1" => (10, 0)
  "(0.0, 1.1),2" => (10, 0)
  "(1.1, 2.1),2" => (10, 0)
  "(1.1, 1.1),2" => (10, 0)
  "(2.1, 1.1),1" => (10, 0)
  "(2.1, 2.1),2" => (10, 0)
  "(0.0, 2.1),1" => (10, 0)
  "(0.0, 1.1),1" => (10, 0)
  "(1.1, 0.0),2" => (10, 0)
  "(2.1, 1.1),2" => (10, 0)
  "(2.1, 0.0),1" => (10, 0)
  "(0.0, 0.0),1" => (10, 0)
  "(2.1, 0.0),2" => (10, 0)

In [30]:
result(ss, n) = Iterators.product(ntuple(i->ss, n)...)
result([1,2,3],3)

Base.Iterators.ProductIterator{Tuple{Array{Int64,1},Array{Int64,1},Array{Int64,1}}}(([1, 2, 3], [1, 2, 3], [1, 2, 3]))

In [ ]:
function init_m(as,cs,n,Ws)
    argumento=fill(as, n-1)
    cartesian_as=collect(Iterators.product(argumento...))
    ms = Dict()
    for a=cartesian_as
        for c=cs
            for my_a=as
                if true  #check if key is there initilize to my_a
                    ms["$(a),$(c)"] = my_a
                end
                
                if true #if already there set to my a if greater.
                    ms["$(a),$(c)"] = my_a
                end
                
        end
    end
    return Ws
end


m0 = init_m(as,cs,n,Ws)
m0(1.1,2)

In [ ]:
function update_Ws(m)
    
end

function generate_ps(m,cs)
end

function update_S(m)
end
    
function update_Js(m)
end



for i in range(10):
    